In [1]:
%%capture
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor


import os
import pandas as pd
import numpy as np


import warnings
import math
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook, show
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
from tqdm.notebook import tqdm
import re
from itertools import repeat

from sklearn import preprocessing

warnings.filterwarnings('ignore')

try:
    %load_ext autotime
    %load_ext jupyterlab_notify
except:
    !pip3 install ipython-autotime
    %load_ext autotime

import imblearn

time: 1.01 s (started: 2023-03-07 18:06:35 -05:00)


## Features

In [2]:
w = [20,14,8]

features = []

player_features = ['ts_pct',
       'efg_pct', 'threepar', 'ftr', 'orb_pct', 'drb_pct', 'trb_pct',
       'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ortg', 'drtg',
       'fg', 'fga', 'fg_pct', 'threep', 'threepa',
       'threep_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'pf', 'pts', 'pts_avg', 'orb_avg', 'drb_avg', 'trb_avg',
       'ast_avg', 'stl_avg', 'blk_avg', 'tov_avg', 'pf_avg']

w = 10
fav_player_ewm_features = []
und_player_ewm_features = []
for i in range(8):
    fav_player_ewm_features += [f'fav_p{i}_{f}_ewm_{w}' for f in player_features]
    und_player_ewm_features += [f'und_p{i}_{f}_ewm_{w}' for f in player_features]

w = 14
fav_player_ewm_fatigue = []
und_player_ewm_fatigue = []
for i in range(8):
    fav_player_ewm_fatigue += [f'fav_p{i}_{f}_ewm_{w}' for f in player_features]
    und_player_ewm_fatigue += [f'und_p{i}_{f}_ewm_{w}' for f in player_features]

streak = ['prev_favorite_win_streak', 
                   'prev_favorite_home_streak', 
                   'prev_underdog_win_streak', 
                   'prev_underdog_home_streak']

elo = ['prev_favorite_elo', 'prev_underdog_elo']
# 5, 8, 12
window_size = 8
ema_favorite_features = \
        [f'prev_favorite_pts_ema{window_size}',     
        f'prev_favorite_bpm_ema{window_size}',      
        f'prev_favorite_fg_ema{window_size}',        
        f'prev_favorite_fg_pct_ema{window_size}',  
        f'prev_favorite_3p_ema{window_size}',       
        f'prev_favorite_3p_pct_ema{window_size}',   
        f'prev_favorite_ft_ema{window_size}',       
        f'prev_favorite_ft_pct_ema{window_size}',    
        f'prev_favorite_orb_ema{window_size}',       
        f'prev_favorite_orb_pct_ema{window_size}',  
        f'prev_favorite_drb_ema{window_size}',      
        f'prev_favorite_drb_pct_ema{window_size}',   
        f'prev_favorite_trb_ema{window_size}',       
        f'prev_favorite_trb_pct_ema{window_size}',   
        f'prev_favorite_tov_ema{window_size}',       
        f'prev_favorite_tov_pct_ema{window_size}',  
        f'prev_favorite_ast_ema{window_size}',       
        f'prev_favorite_ast_pct_ema{window_size}',   
        f'prev_favorite_stl_ema{window_size}',       
        f'prev_favorite_stl_pct_ema{window_size}',   
        f'prev_favorite_blk_ema{window_size}',       
        f'prev_favorite_blk_pct_ema{window_size}',   
        f'prev_favorite_drtg_ema{window_size}',      
        f'prev_favorite_ortg_ema{window_size}',      
        f'prev_favorite_efg_pct_ema{window_size}',  
        f'prev_favorite_pace_ema{window_size}']

ema_underdog_features = [f.replace('favorite','underdog') for f in ema_favorite_features]

features += fav_player_ewm_features + und_player_ewm_features + \
            streak + elo + ema_favorite_features + ema_underdog_features
 

time: 1.4 ms (started: 2023-03-07 18:06:36 -05:00)


# Oversampling

In [3]:
df = pd.read_csv("nba_processed_features + elo.csv")

df.dropna(inplace=True)

time: 5.15 s (started: 2023-03-07 18:06:36 -05:00)


In [4]:
def train_test_split(data):
    cutoff = math.floor(len(data) * 0.7)
    train = data[data.index < cutoff].copy()
    test = data[data.index >= cutoff].copy()
    return train, test 

time: 280 µs (started: 2023-03-07 18:06:41 -05:00)


In [5]:
from imblearn.over_sampling import RandomOverSampler
mm_scaler = preprocessing.MinMaxScaler()

train, test = train_test_split(df)

X_train = train[features]
X_test = test[features]

X_train = mm_scaler.fit_transform(X_train)
X_test = mm_scaler.fit_transform(X_test)

y_train = train['favorite_won']
y_test = test['favorite_won']

# oversample training data
oversample = RandomOverSampler(sampling_strategy='minority')
X_train, y_train = oversample.fit_resample(X_train, y_train)





time: 172 ms (started: 2023-03-07 18:06:41 -05:00)


In [6]:
# Creating np arrays
train_target = torch.tensor(y_train)
train_target = train_target.to(torch.float32)
train_target = train_target.unsqueeze(1)
print(train_target.dtype)

train_features = torch.tensor(X_train)
train_features = train_features.to(torch.float32)

test_target = torch.tensor(y_test.values)
test_target = test_target.to(torch.float32)
test_target = test_target.reshape(-1,1)unsqueeze(1)


test_features = torch.tensor(X_test)
test_features = test_features.to(torch.float32)

# Passing to DataLoader
train = TensorDataset(train_features, train_target)
test = TensorDataset(test_features, test_target)


torch.float32
time: 48.1 ms (started: 2023-03-07 18:06:42 -05:00)


In [7]:
batch_size = 64
# Create data loaders.
train_dataloader = DataLoader(train, batch_size=batch_size)
test_dataloader = DataLoader(test, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
print(y.dtype)

Shape of X [N, C, H, W]: torch.Size([64, 714])
Shape of y: torch.Size([64, 1]) torch.float32
torch.float32
time: 63.8 ms (started: 2023-03-07 18:06:42 -05:00)


In [8]:
# # Get cpu or gpu device for training.
# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
# print(f"Using {device} device")

# # Define model
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.layer1 = nn.Linear(714, 714)
#         self.act1 = nn.ReLU()
#         self.layer2 = nn.Linear(714, 714)
#         self.act2 = nn.ReLU()
#         self.layer3 = nn.Linear(714, 714)
#         self.act3 = nn.ReLU()
#         self.output = nn.Linear(714, 1)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         x = self.act1(self.layer1(x))
#         x = self.act2(self.layer2(x))
#         x = self.act3(self.layer3(x))
#         x = self.sigmoid(self.output(x))
#         return x
    
# model = NeuralNetwork().to(device)
# print(model)

time: 236 µs (started: 2023-03-07 18:06:42 -05:00)


In [9]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(714, 714)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(714, 714)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(714, 714)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(714, 1)

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.output(x)
        return x
    
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (layer1): Linear(in_features=714, out_features=714, bias=True)
  (act1): ReLU()
  (layer2): Linear(in_features=714, out_features=714, bias=True)
  (act2): ReLU()
  (layer3): Linear(in_features=714, out_features=714, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=714, out_features=1, bias=True)
)
time: 555 ms (started: 2023-03-07 18:06:42 -05:00)


In [10]:
import torch.optim as optim

loss_fn = nn.BCELoss()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

time: 446 µs (started: 2023-03-07 18:06:42 -05:00)


In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print(size)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            #_, pred = torch.max(pred, dim=1)
            test_loss += loss_fn(pred, y).item()
            #print(pred.argmax(1))
            #class_pred = torch.round(y_test_pred)
            #print(class_pred)
            #correct += (
    #test_loss /= num_batches
   # print(correct)
    #correct /= size
    #print(correct)
    #print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


time: 700 µs (started: 2023-03-07 18:06:42 -05:00)


In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        #y = y.unsqueeze(1)
        # Compute prediction error
        pred = model(X)
        #_, pred = torch.max(pred, dim=1)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

time: 351 µs (started: 2023-03-07 18:06:42 -05:00)


In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.690522  [   64/17022]
loss: 0.680134  [ 6464/17022]
loss: 0.772090  [12864/17022]
6139
Epoch 2
-------------------------------
loss: 1.362254  [   64/17022]
loss: 0.675594  [ 6464/17022]
loss: 0.763507  [12864/17022]
6139
Epoch 3
-------------------------------
loss: 2.529950  [   64/17022]
loss: 0.690276  [ 6464/17022]
loss: 0.886971  [12864/17022]
6139
Epoch 4
-------------------------------
loss: 0.699502  [   64/17022]
loss: 0.690025  [ 6464/17022]
loss: 0.755231  [12864/17022]
6139
Epoch 5
-------------------------------
loss: 0.709486  [   64/17022]
loss: 0.691630  [ 6464/17022]
loss: 0.736618  [12864/17022]
6139
Done!
time: 1.97 s (started: 2023-03-07 18:07:40 -05:00)


In [23]:
model.train()
train_loss = []

#Loss Computation
loss_func = nn.BCEWithLogitsLoss()
#Optimizer
learning_rate = 0.1
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 5


for epoch in range(epochs):
    #Within each epoch run the subsets of data = batch sizes.
    for xb, yb in train_dataloader:
        xb, yb = xb.to(device), yb.to(device)
        y_pred = model(xb)            # Forward Propagation
        loss = loss_fn(y_pred, yb)  # Loss Computation
        optimizer.zero_grad()         # Clearing all previous gradients, setting to zero 
        loss.backward()               # Back Propagation
        optimizer.step()              # Updating the parameters 
    #print("Loss in iteration :"+str(epoch)+" is: "+str(loss.item()))
    train_loss.append(loss.item())
print('Last iteration loss value: '+str(loss.item()))

Last iteration loss value: 0.017957134172320366
time: 1.84 s (started: 2023-03-07 18:11:01 -05:00)


In [24]:
import itertools

y_pred_list = []
model.eval()
#Since we don't need model to back propagate the gradients in test set we use torch.no_grad()
# reduces memory usage and speeds up computation
with torch.no_grad():
    for xb_test,yb_test  in test_dataloader:
        xb_test, yb_test = xb_test.to(device), yb_test.to(device)
        y_test_pred = model(xb_test)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().detach().numpy())

#Takes arrays and makes them list of list for each batch        
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
#flattens the lists in sequence
ytest_pred = list(itertools.chain.from_iterable(y_pred_list))


time: 59.7 ms (started: 2023-03-07 18:11:03 -05:00)


In [26]:
from sklearn.metrics import confusion_matrix
y_true_test = y_test
conf_matrix = confusion_matrix(y_true_test ,ytest_pred)
print("Confusion Matrix of the Test Set")
print("-----------")
print(conf_matrix)
# print("Precision of the MLP :\t"+str(precision_score(y_true_test,ytest_pred)))
# print("Recall of the MLP    :\t"+str(recall_score(y_true_test,ytest_pred)))
# print("F1 Score of the Model :\t"+str(f1_score(y_true_test,ytest_pred)))

Confusion Matrix of the Test Set
-----------
[[   0    0    0]
 [1999    0    0]
 [4140    0    0]]
time: 6.21 ms (started: 2023-03-07 18:11:09 -05:00)


In [ ]:
criterion = nn.BCEWithLogitsLoss()
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch)
        acc = binary_acc(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

In [ ]:
import tqdm
import torch.optim as optim
def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    n_epochs = 250   # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

In [ ]:
import torch.nn as nn

class Wide(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(741, 180)
        self.relu = nn.ReLU()
        self.output = nn.Linear(180, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split

# create model, train, and get accuracy
model = NeuralNetwork()
acc = model_train(model, X_train[train], y_train[train], X_train[test], y_train[test])
print("Accuracy (wide): %.2f" % acc)
